In [1]:
// Defining input
class TransVal()
case class Transaction(assignedVar: TransVal, readVars: List[TransVal]) {
  def isDependentOn(other: Transaction): Boolean = {
    other.readVars.contains(this.assignedVar) ||
      this.readVars.contains(other.assignedVar) ||
      this.assignedVar == other.assignedVar
  }

  def isIndependentOn(other: Transaction): Boolean = !isDependentOn(other)
}

val X = new TransVal()
val Y = new TransVal()
val Z = new TransVal()
val V = new TransVal()

val A = Transaction(X, List(X, Y))
val B = Transaction(Y, List(Y, Z))
val C = Transaction(X, List(X, Z))
val D = Transaction(Z, List(Y, Z))

val ops = Array(A, B, C, D)

val word = Array(B,A,A,D,C,B)

defined class TransVal
defined class Transaction
X: $user.TransVal = cmd0$$user$TransVal@49b8826
Y: $user.TransVal = cmd0$$user$TransVal@592544b7
Z: $user.TransVal = cmd0$$user$TransVal@19185220
V: $user.TransVal = cmd0$$user$TransVal@21f541b3
A: $user.Transaction = Transaction(
  cmd0$$user$TransVal@49b8826,
  List(cmd0$$user$TransVal@49b8826, cmd0$$user$TransVal@592544b7)
)
B: $user.Transaction = Transaction(
  cmd0$$user$TransVal@592544b7,
  List(cmd0$$user$TransVal@592544b7, cmd0$$user$TransVal@19185220)
)
C: $user.Transaction = Transaction(
  cmd0$$user$TransVal@49b8826,
  List(cmd0$$user$TransVal@49b8826, cmd0$$user$TransVal@19185220)
)
D: $user.Transaction = Transaction(
  cmd0$$user$TransVal@19185220,
  List(cmd0$$user$TransVal@592544b7, cmd0$$user$TransVal@19185220)
)
ops: Array[$user.Transaction] = Array(
  Transaction(
    cmd0$$user$TransVal@49b8826,
    List(cmd0$$user$TransVal@49b8826, cmd0$$user$TransVal@592544b7)
  ),
  Transaction(
    cmd0$$user$TransVal@592544b7,
   

In [2]:
// deps
val D = ops.map(i => ops.map(o => i.isDependentOn(o)))

D: Array[Array[Boolean]] = Array(
  Array(true, true, true, false),
  Array(true, true, false, true),
  Array(true, false, true, true),
  Array(false, true, true, true)
)

In [3]:
val I = ops.map(i => ops.map(o => i.isIndependentOn(o)))

I: Array[Array[Boolean]] = Array(
  Array(false, false, false, true),
  Array(false, false, true, false),
  Array(false, true, false, false),
  Array(true, false, false, false)
)

In [4]:
class Step(val t: Transaction, val i: Integer)

defined class Step

In [5]:

val ww = word.zipWithIndex.map(x => new Step(x._1, x._2))


ww: Array[Step] = Array(
  cmd3$$user$Step@7b826543,
  cmd3$$user$Step@2da2e31c,
  cmd3$$user$Step@71713e07,
  cmd3$$user$Step@19a269af,
  cmd3$$user$Step@7162afa5,
  cmd3$$user$Step@4be20896
)

In [6]:
// initial monad 
val nonqueue = ww.map(w => (w,0))

nonqueue: Array[(Step, Int)] = Array(
  (cmd3$$user$Step@7b826543, 0),
  (cmd3$$user$Step@2da2e31c, 0),
  (cmd3$$user$Step@71713e07, 0),
  (cmd3$$user$Step@19a269af, 0),
  (cmd3$$user$Step@7162afa5, 0),
  (cmd3$$user$Step@4be20896, 0)
)

In [7]:
def stackify(ar:Array[(Step, Int)], v: Step) : Array[(Step, Int)] = {
    ar.map(o => o match {
        case o if (o._1.i < v.i) => o
        case o if o._1 == v  => o
        case o if o._1.t.isDependentOn(v.t) => (o._1, o._2 + 1)
        case o => o
    })
}

defined function stackify

In [8]:
// numder of blocking ops before
val qued = ww.foldLeft(nonqueue)((acc, i) => stackify(acc, i))

qued: Array[(Step, Int)] = Array(
  (cmd3$$user$Step@7b826543, 0),
  (cmd3$$user$Step@2da2e31c, 1),
  (cmd3$$user$Step@71713e07, 2),
  (cmd3$$user$Step@19a269af, 1),
  (cmd3$$user$Step@7162afa5, 3),
  (cmd3$$user$Step@4be20896, 4)
)

In [9]:
def destackify(ar:Array[(Step, Int)]) : (Array[Step], Array[(Step, Int)]) = {
    val fl = ar.filter(o => o._2 == 0).map(g => g._1)
    val ot = ar.filter(o => o._2 > 0)
    val ott = ot.map(o => (o._1, o._2 - fl.count(f => f.t.isDependentOn(o._1.t) && f.i < o._1.i)))
    (fl, ott)
}

defined function destackify

In [10]:
def prepare(a : Array[(Step, Int)], ff: Array[Array[Step]]):Array[Array[Step]] = {
    if (a.length == 0) {
        //sorry
        return ff
    } else {
        val h = destackify(a)
        return prepare(h._2, ff:+h._1)
    }
}



defined function prepare

In [11]:
val y = prepare(qued, Array.empty[Array[Step]])

y: Array[Array[Step]] = Array(
  Array(cmd3$$user$Step@7b826543),
  Array(cmd3$$user$Step@2da2e31c, cmd3$$user$Step@19a269af),
  Array(cmd3$$user$Step@71713e07),
  Array(cmd3$$user$Step@7162afa5, cmd3$$user$Step@4be20896)
)

In [47]:
// Ex 1. Floata
y.map(u => u.map(s => s.i))
// Yay

res41: Array[Array[Integer]] = Array(Array(0), Array(1, 3), Array(2), Array(4, 5))

In [12]:
val deps = ww.map(w => ww.map(o => w.t.isDependentOn(o.t) && w.i > o.i))

deps: Array[Array[Boolean]] = Array(
  Array(false, false, false, false, false, false),
  Array(true, false, false, false, false, false),
  Array(true, true, false, false, false, false),
  Array(true, false, false, false, false, false),
  Array(false, true, true, true, false, false),
  Array(true, true, true, true, false, false)
)

In [13]:
import scala.collection.mutable.ArrayBuffer

var isAnythingChanged : Boolean = true
val D2 = ArrayBuffer[ArrayBuffer[Boolean]]()
for (y1 <- deps.indices) {
    D2 += ArrayBuffer[Boolean]()
    for (x1 <- deps(y1).indices) {
        D2(y1) += false;
    }
}



while (isAnythingChanged) {
    val cp = D2.map(o => o.map(x => x))
    for (y1 <- deps.indices) {
        for (x1 <- deps(y1).indices) {
            for (w <- deps.indices) {
                D2(y1)(x1) = D2(y1)(x1) || (deps(y1)(w) && deps(w)(x1) && y1 > w && w > x1)
            }
        }
    }
    
    isAnythingChanged = false
    
    for (y1 <- deps.indices) {
        for (x1 <- deps(y1).indices) {
            isAnythingChanged = isAnythingChanged || D2(y1)(x1) != cp(y1)(x1)
        }
    }
}



import scala.collection.mutable.ArrayBuffer
isAnythingChanged: Boolean = false
D2: collection.mutable.ArrayBuffer[collection.mutable.ArrayBuffer[Boolean]] = ArrayBuffer(
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(true, false, false, false, false, false),
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(true, true, false, false, false, false),
  ArrayBuffer(true, true, false, false, false, false)
)

In [14]:
val indirDeps = ArrayBuffer[ArrayBuffer[Boolean]]()
for (y1 <- deps.indices) {
    indirDeps += ArrayBuffer[Boolean]()
    for (x1 <- deps(y1).indices) {
        indirDeps(y1) += false;
    }
}

for (y1 <- deps.indices) {
    for (x1 <- deps(y1).indices) {
        indirDeps(y1)(x1) = deps(y1)(x1) && !D2(y1)(x1)
    }
}

indirDeps: ArrayBuffer[ArrayBuffer[Boolean]] = ArrayBuffer(
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(true, false, false, false, false, false),
  ArrayBuffer(false, true, false, false, false, false),
  ArrayBuffer(true, false, false, false, false, false),
  ArrayBuffer(false, false, true, true, false, false),
  ArrayBuffer(false, false, true, true, false, false)
)

In [15]:
// ex 2
println("http://www.webgraphviz.com/")
println("digraph G {")
for (y1 <- indirDeps.indices) {
    for (x1 <- indirDeps(y1).indices) {
        if (indirDeps(y1)(x1)) {
            println("  " + x1 + " -> " + y1)
        }
    }
}
println("}")

http://www.webgraphviz.com/
digraph G {
  0 -> 1
  1 -> 2
  0 -> 3
  2 -> 4
  3 -> 4
  2 -> 5
  3 -> 5
}


In [16]:
// ex 3
var pardeps = ArrayBuffer[(Int, Int)]()
for (y1 <- indirDeps.indices) {
    for (x1 <- indirDeps(y1).indices) {
        if (indirDeps(y1)(x1)) {
            pardeps += ((x1, y1))
        }
    }
}

var wds = ww.map(w => w.i)

val printed = ArrayBuffer[Int]()

var doItOnceMore = true

while (pardeps.length != 0 || doItOnceMore){
    if (pardeps.length == 0) {
        doItOnceMore = false
    }
    var floatable = wds.filter(w => pardeps.count(p => p._2 == w) == 0 && !printed.contains(w))
    println("")
    print("[")
    for (y1 <- floatable) {
        printed += y1;
        print(y1 + " ")
    }
    print("]")
    println("")
    pardeps = pardeps.filter(p => floatable.count(w => w == p._1) == 0)
}



[0 ]

[1 3 ]

[2 ]

[4 5 ]


pardeps: ArrayBuffer[(Int, Int)] = ArrayBuffer()
wds: Array[Integer] = Array(0, 1, 2, 3, 4, 5)
printed: ArrayBuffer[Int] = ArrayBuffer(0, 1, 3, 2, 4, 5)
doItOnceMore: Boolean = false